In [10]:
from shapely.geometry import box
import argparse
import geopandas as gpd
import pandas as pd
import os
import glob
from tqdm import tqdm
import numpy as np
# from solaris.eval.iou import calculate_iou
from solaris.eval.vector import mF1

In [25]:
input_folder = r"C:\Users\Professional\yoloplanes\yolo\inference\class_one_out\bounding_boxes"
output_folder = r"C:\Users\Professional\yoloplanes\wdata\test\geojson_aircraft_bbox"
os.makedirs(output_folder,exist_ok=True)
os.chdir(input_folder)
geojsons = glob.glob("*.geojson")
for geojson in tqdm(geojsons):
    gdf = gpd.read_file(geojson)
    box_geoms= []
    for _, row in gdf.iterrows():
        x = row.geometry.bounds
        bbox = box(x[0], x[1], x[2], x[3])
        box_geoms.append(bbox)
    gdf['box_geom'] = box_geoms
    gdf = gdf.drop('geometry',axis=1)
    gdf = gpd.GeoDataFrame(gdf, geometry='box_geom')
    out_name=os.path.join(output_folder, geojson)
    gdf.to_file(out_name, driver='GeoJSON', encoding='utf-8')

100%|██████████| 66/66 [00:03<00:00, 16.88it/s]


In [26]:
def count_unique_index(df, by):
    return df.groupby(by).size().reset_index().rename(columns={0: 'count'})

def create_custom_classes(all_annotations_geojson, geojson_dir, output_path, category_attributes):
    os.makedirs(output_path, exist_ok=True)
    gdf = gpd.read_file(all_annotations_geojson)
    lookup_gdf = count_unique_index(gdf, category_attributes)
    lookup_gdf['custom_id'] = list(range(0, len(lookup_gdf)))
    lookup_gdf.drop(columns=['count'], inplace=True)
    lookup_gdf.to_csv(os.path.join(output_path, "custom_class_lookup.csv"))
    os.chdir(geojson_dir)
    geojsons = glob.glob("*.geojson")
    for geojson in tqdm(geojsons):
        gdf = gpd.read_file(geojson)
        gdf = pd.merge(gdf, lookup_gdf, on=category_attributes, how='left')
        gdf["custom_id"] = pd.to_numeric(gdf["custom_id"], downcast='float')
        gdf.to_file(os.path.join(output_path, geojson), driver="GeoJSON", encoding='utf-8')

In [27]:
all_annotations_geojson = r'C:\Users\Professional\yoloplanes\wdata\RarePlanes_Public_All_Annotations.geojson'
geojson_dir_test = r'C:\Users\Professional\yoloplanes\wdata\test\geojson_aircraft_bbox'

output_path_test_one = r'C:\Users\Professional\yoloplanes\wdata\test\yolo_class_one_truth_bbox'

class_one = ['propulsion']

create_custom_classes(all_annotations_geojson, geojson_dir_test, output_path_test_one, class_one)

  0%|          | 0/66 [00:00<?, ?it/s]


KeyError: 'propulsion'

In [28]:
proposal_polygons_dir =r"C:\Users\Professional\yoloplanes\yolo\inference\class_one_out\bounding_boxes"
gt_polygons_dir = r"C:\Users\Professional\yoloplanes\wdata\test\yolo_class_one_truth_bbox"
mF1, f1s_by_class, precision_iou_by_obj, precision_by_class, mPrecision, recall_iou_by_obj, recall_by_class, mRecall, object_subset, prop_objs, all_objs = mF1(proposal_polygons_dir, gt_polygons_dir, prediction_cat_attrib="class_id", gt_cat_attrib='custom_id', object_subset=[], all_outputs=True)

getting unique objects...


100%|██████████| 2710/2710 [00:00<00:00, 52254.78it/s]


calculating recall...


  9%|▊         | 237/2710 [00:01<00:19, 126.06it/s]


KeyboardInterrupt: 

In [16]:
lookup_table = r"C:\Users\Professional\yoloplanes\geojsons_test\yolo_class_one\custom_class_lookup.csv"
lookup_table = pd.read_csv(lookup_table)
lookup_table.drop(lookup_table.columns[lookup_table.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
object_subset_int = [int(i) + 1 for i in [0,1,2]]
out_dict = {"custom_id":object_subset_int,"f1":f1s_by_class,"precision": precision_by_class, "recall": recall_by_class}
out_df = pd.DataFrame.from_dict(out_dict)
out_df = pd.merge(out_df, lookup_table, on='custom_id', how='left')
out_df

NameError: name 'f1s_by_class' is not defined

Now, using these fused geojsons, you can visulaize the predictions by downloading the `/home/ubuntu/src/yolo_planes/yolov5/inference/class_one_out/bounding_boxes` folder to your local machine and using a geographic infromation system like QGIS and overlaying the bounding boxes onto the real image.